# Data Cleaning and Machine Learning 

## Reading the New Integrated Sample

In [26]:
import numpy as np
import imdb 
import pandas as pd
anime_data = pd.read_csv("C://Users//Master//new_anime_data1.csv")

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


## Data Transformation

### Drop Some Special Characters from "genre" and "overview", and Covert "type" to Movie/Tv series

In [27]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
anime_data.drop_duplicates(inplace = True)

### Applying One-Hot and Binary Encoding

In [28]:
from sklearn.preprocessing import LabelBinarizer

anime_data = anime_data.dropna()

#one-hot encoding to transform the genres column to numerical columns
df = anime_data.genre.str.get_dummies(',')

#binary encoding for type column
type_lb = LabelBinarizer()
X = type_lb.fit_transform(anime_data.type.values)


dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
anime_data = pd.concat([anime_data, dfOneHot], axis=1)
anime_data = pd.concat([anime_data, df], axis=1)

anime_data['movie/TVseries'].fillna(0, inplace=True)

#anime_data

### Counting words "overview"

In [29]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    print(result)
    return result

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

summary_doc = anime_data['overview'].apply(clean_words)
summary_doc =summary_doc.apply(','.join)
 
vectorizer = TfidfVectorizer()
overview_feature = vectorizer.fit_transform(summary_doc).toarray()
overview_feature = vectorizer.fit_transform(summary_doc)


df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
anime_data = pd.concat([anime_data, df], axis=1)

#drop Null values
anime_data = anime_data.dropna(inplace=True)

["'Best", 'friends', 'Mikan', 'Hotaru', 'Hotaru', 'special', 'school', 'Alice', 'Academy', 'Mikan', 'determines', 'too', 'Alice', 'Academy', 'mysterious', 'place']
["'Best", 'friends', 'Mikan', 'Hotaru', 'Hotaru', 'special', 'school', 'Alice', 'Academy', 'Mikan', 'determines', 'too', 'Alice', 'Academy', 'mysterious', 'place']
['Lundgren', 'Jack', 'Devlin', 'U.S', 'Marshal', 'high-profile', 'clients', 'matter', 'security', ':DropZone', '<', 'peeri', '@', '>', 'Casey', 'daughter', 'friend', 'assassins', 'Jack', 'Devlin', 'strange', 'phobia', 'fear', 'white', 'color', 'other', 'friend', 'bodyguard', 'supermodel', 'Jack', 'Now', 'fear', 'assassin', 'well', 'aware', 'Jack', 'problem', ':Boris', 'Shafir', '<', '@', 'hsi.com', '>']
['Lundgren', 'Jack', 'Devlin', 'U.S', 'Marshal', 'high-profile', 'clients', 'matter', 'security', ':DropZone', '<', 'peeri', '@', '>', 'Casey', 'daughter', 'friend', 'assassins', 'Jack', 'Devlin', 'strange', 'phobia', 'fear', 'white', 'color', 'other', 'friend', 'b

TypeError: expected string or bytes-like object

In [9]:
'''
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
overview_feature = vectorizer.fit_transform(anime_data['overview']).toarray()

df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
anime_data = pd.concat([anime_data, df], axis=1)
'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\nvectorizer = CountVectorizer()\noverview_feature = vectorizer.fit_transform(anime_data[\'overview\']).toarray()\n\ndf = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])\nanime_data = pd.concat([anime_data, df], axis=1)\n'

## Feature Extraction

In [31]:
anime_data = anime_data.drop(columns=['anime_id', 'name', 'genre', 'overview', 'type'])
anime_data

,Unnamed: 0,episodes,rating,members,movie/TVseries,Action,Adventure,Animation,Biography,Cars,...,Sports,SuperPower,Supernatural,Talk-Show,Thriller,Vampire,War,Western,Yaoi,Yuri
0,3009.0,50,7.11,1009.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5694.0,2,6.38,2708.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5351.0,4,6.48,748.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1056.0,1,7.70,24880.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5383.0,2,6.47,265.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7999.0,1,4.15,195.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,184.0,25,8.35,246899.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8599.0,1,6.50,95.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,6394.0,13,6.14,7778.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2474.0,1,7.25,1327.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Replace Null Values With the Most Frequent Value

In [25]:
'''
from sklearn.impute import SimpleImputer

#anime_data.fillna(value=pd.np.nan, inplace=True)
imp = SimpleImputer(strategy="most_frequent")
feature_vect = imp.fit_transform(anime_data)

df = pd.DataFrame(feature_vect, columns = ["feature"+ str(int(i)) for i in range(feature_vect.shape[1])])
anime_data = df
'''

### Replace Null Values with Mean

In [32]:
from sklearn.impute import SimpleImputer

#anime_data.fillna(value=pd.np.nan, inplace=True)
imp = SimpleImputer(strategy="mean")
feature_vect = imp.fit_transform(anime_data)

df = pd.DataFrame(feature_vect, columns = ["feature"+ str(int(i)) for i in range(feature_vect.shape[1])])
anime_data = df
anime_data

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature52,feature53,feature54,feature55,feature56,feature57,feature58,feature59,feature60,feature61
0,3009.0,50.0,7.11,1009.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5694.0,2.0,6.38,2708.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5351.0,4.0,6.48,748.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1056.0,1.0,7.70,24880.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5383.0,2.0,6.47,265.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7999.0,1.0,4.15,195.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,184.0,25.0,8.35,246899.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8599.0,1.0,6.50,95.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,6394.0,13.0,6.14,7778.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2474.0,1.0,7.25,1327.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Modeling

In [33]:
from sklearn import preprocessing

y = anime_data['feature2']
#lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = y #lab_enc.fit_transform(y)

X = anime_data.drop(columns=['feature2'])

# Split the data into training/testing sets
anime_X_train = X[:-500]
anime_X_test = X[-500:]

# Split the targets into training/testing sets
anime_y_train = training_scores_encoded[:-500]
anime_y_test = training_scores_encoded[-500:]

### Generate Random Values, add it to "members" Attribute

In [152]:
'''
import warnings
warnings.filterwarnings("ignore")

n_samples = anime_X_train.shape[0]
n_features = anime_X_train.shape[1]

rand_rate = 0.01
n_rand_samples = int(np.floor(n_samples * rand_rate))

mu, sigma = anime_X_train['feature3'].mean(), anime_X_train['feature3'].std()

anime_X_train['feature3'].iloc[0:n_rand_samples] = abs(np.random.normal(mu, sigma, size=n_rand_samples))
'''



### Training and Testing

In [34]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(anime_X_train, anime_y_train)

# Make predictions using the testing set
anime_y_pred = regr.predict(anime_X_test)


## Evaluation

In [35]:
#accurcy
print('Accurcy: \n', regr.intercept_)

# The coefficients
print('Coefficients: \n', regr.coef_)

# The mean squared error
print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(anime_y_test, anime_y_pred))



7.585691218359104
Coefficients: 
 [-1.82042122e-04  4.54335793e-04  2.34452048e-06 -5.78299787e-02
  1.88944653e-02 -1.08409209e-02  3.46245770e-02  3.05026250e-02
 -1.29215211e-01 -5.96481927e-02  3.27283466e-02  2.35766802e-02
  6.97060803e-03  3.47684492e-02 -2.89355146e-03  1.31166885e-01
 -1.66722964e-01  8.43699603e-02 -1.15974369e-02 -1.35816164e-01
 -1.38964343e-01 -1.51133369e-01 -2.07484103e-01 -5.37943912e-02
 -5.68507373e-02 -1.66684291e-01 -2.51621908e-02  1.63275333e-01
 -1.12475936e-01  3.59021983e-02 -1.11049186e-01  1.47058094e-01
 -9.08228165e-02 -2.01572798e-01  1.78296062e-01 -8.29235069e-02
  7.02332147e-02  1.71309908e-01  5.79186351e-02 -7.05258596e-03
  5.31979862e-01  2.09991547e-01 -5.62164939e-02 -2.33840278e-01
 -1.08865571e-01 -1.33730382e-01 -5.88525393e-02  6.55099101e-01
 -3.98183077e-02 -9.75134557e-02  1.85650719e-01  2.86668231e-01
 -2.69496785e-02  6.35004898e-02 -8.64325209e-02  1.76245832e-01
  4.95491205e-01 -1.67151228e-01 -1.10736020e-01 -2.0007